In [0]:
#불용어 제거
def remove_stop_words(corpus):
    stop_words = ['is', 'a', 'will', 'be']
    results = []
    for text in corpus:
        tmp = text.split(' ')
        for stop_word in stop_words:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))
    
    return results

In [0]:
corpus = remove_stop_words(corpus)
corpus

### word2vec table 생성

In [0]:
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize #단어토큰화
from gensim.models import Word2Vec #word2vec
from gensim.models import KeyedVectors
# for text in corpus:
#     print(text)

df = pd.DataFrame(data, columns = ['input', 'label'])
result = [word_tokenize(text) for text in corpus]

In [0]:
#size=w1,w2...의 개수
#window는 주변 몇개의 단어까지 고려할 것인가
model = Word2Vec(sentences=result, size=2, window=2, min_count=1, workers=4, sg=1)
model_result = model.wv.most_similar("man")
print(model_result)

In [0]:
#모델저장하기
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") #불러오기

In [0]:
#모델불러오기
model_result = loaded_model.most_similar("man")
print(model_result)

### kakao arena에 해보기



> #### 모듈 불러오기/설치
 - khaiii는 우분투 환경에서만 가능
 - 사전 추가할 경우에는 khaiii 다시 build 해야함
    - 다시 build하면 시간이 오래 걸려서 한번 추가할 때 몰아서 하는게 효율적일 듯




In [0]:
import pickle
import pandas as pd
import sys
import re
from gensim.models import Word2Vec


In [0]:
#khaiii 설치
!git clone https://github.com/kakao/khaiii.git

!pip install cmake

!mkdir build

!cd build && cmake /content/khaiii

!cd /content/build/ && make all

!cd /content/build/ && make resource

!cd /content/build && make install

!cd /content/build && make package_python

!pip install /content/build/package_python

from khaiii import KhaiiiApi
api = KhaiiiApi()




> ### tag & playlist 같이 있는 데이터 만들기
- 한 행에 playlist 이름이랑 tag를 합쳤음
    - 이때 특수문자나 자음, 모음만 있는거는 모두 날림.
    - ex) '짝사랑 고백 사랑 이별 슬픔 감성을 자극하는 곡들 짝사랑 취향저격 슬픔 고백 사랑 이별'
- output.txt로 저장시킴


In [0]:
# tag & playlist 하나로 합치고, cleansing 해서 txt파일로 저장
with open('train_genre.pickle', 'rb') as f:
    train = pickle.load(f)

def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    return text  

i=0
import sys
text = open('output.txt','w')

while i < len(train):
    mylist = [[train.plylst_title[i]],train.tags[i],train.genre[i]]
    clean = clean_str(' '.join(sum(mylist,[])))
    text.write(clean+'\n')
    print(clean)
    i+=1
text.close()

with open('output.txt') as f:
    data = f.read().splitlines()

In [77]:
# ex = '그의 화려한 화려하는 바보이다'
# re.sub(r"(의|한|하는|이다)\b|그의", "",ex)


'그 화려 화려 바보'

In [48]:
#원하는 파일/디렉토리 명 찾고싶을 때
!find / -name 'char_align.map*'

/content/build/_CPack_Packages/Linux-Source/ZIP/khaiii-0.4/rsc/src/char_align.map
/content/khaiii/rsc/src/char_align.map




> ### khaiii 사전추가
- 사전추가 할 때 다시 build 하는 코드



In [0]:
#사전추가
#!cd /content/khaiii/rsc # 어디에 있던 rsc로 와서 실행하기
#!mkdir -p ../build/share/khaiii
#!PYTHONPATH=/content/khaiii/src/main/python/ /content/khaiii/rsc/bin/compile_preanal.py --rsc-src=/content/khaiii/rsc/src --rsc-dir=/content/build/share/khaiii

!cd /content/build/ && make resource

!cd /content/build && make install

!cd /content/build && make package_python

!pip install /content/build/package_python

In [64]:
#사전추가 잘 됐는지 확인용(캐럴)
from khaiii import KhaiiiApi
api = KhaiiiApi()
for word in api.analyze(data[3]):
    print(word)

크리스마스	크리스마스/NNG
분위기에	분위기/NNG + 에/JKB
흠뻑	흠뻑/MAG
취하고	취하/VV + 고/EC
싶을때	싶/VX + 을/ETM + 때/NNG
연말	연말/NNG
눈오는날	눈/NNG + 오/VV + 는/ETM + 날/NNG
캐럴	캐럴/NNG
분위기	분위기/NNG
따듯한	따듯/XR + 하/XSA + ㄴ/ETM
크리스마스캐럴	크리스마스캐럴/NNG
겨울노래	겨울/NNG + 노래/NNG
크리스마스	크리스마스/NNG
겨울왕국	겨울왕국/NNG
크리스마스송	크리스마스/NNG + 송/MAG




> ### khaiii 형태소 추출
 - 일단 명사만 추출하여 morphs에 저장
 - 이중리스트 형태
    - 제일 하부 리스트는 한 플레이리스트에 대한 명사임.



In [0]:
#띄어쓰기 안되어있어서 이거만 일단 수작업으로 바꿔둠
data[6] = '짝사랑 고백 사랑 이별 슬픔 감성을 자극하는 곡들 짝사랑 취향저격 슬픔 고백 사랑 이별'

In [0]:
#토큰화하기
morphs = []
tmp_list = []
for i in range(0,len(data)):
    for word in api.analyze(data[i]):
        for morph in word.morphs:
            if morph.tag in ['NNG', 'NNP','NP']:
                tmp_list.append(morph.lex)
    morphs.append(tmp_list)
    tmp_list = []




> ### word2vec
- 내 생각: 한 자리 단어, 비슷한 단어 등을 묶어 주고 다시 해봐야할 듯. 지금 단어를 넣으면 비슷한 말들을 뽑아줘서 정작 tag를 안보여줌.
    - ex) '요즘'이랑 비슷한 단어 검색하면 '자주', '요새'가 나옴. 이건 너무 당연한 말이라서 분석이라고 보기가 힘듦. 이런 단어들 전처리 필요

- 저번에 블로그에서 봤던거 처럼 가중치 행렬을 만들어서 문장을 넣어도 태그를 예측할 수 있게끔 해도 좋을듯.
https://ratsgo.github.io/natural%20language%20processing/2017/03/08/word2vec/



In [155]:
# word2vec with playlist_title
from gensim.models import Word2Vec
model_play = Word2Vec(sentences=morphs,
                 size=256, window=4,
                 min_count=2,
                 workers=4,
                 sg=1)
print(model_play)

#model_play.save('play_w2v.model') # 모델 저장

Word2Vec(vocab=14812, size=256, alpha=0.025)


In [166]:
model_result = model_play.wv.most_similar("요즘")
print(model_result)

[('요새', 0.7125931978225708), ('유행', 0.6676451563835144), ('자주', 0.663533627986908), ('목록', 0.6608127355575562), ('최애', 0.6597887277603149), ('길거리', 0.6588185429573059), ('핫한', 0.6538964509963989), ('위주', 0.6538501977920532), ('한곡', 0.6501908898353577), ('내맘', 0.6498870253562927)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [161]:
train.head(13)

,id,like_cnt,plylst_title,songs,tags,updt_date,genre
0,61281,71,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",[락],2013-12-19 18:36:19.000,"[J-POP, 일렉트로니카, R&B/Soul, POP, 포크/블루스/컨트리, 록/메탈]"
1,10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[추억, 회상]",2014-12-02 16:19:42.000,"[포크/블루스, 일렉트로니카, 재즈, 뉴에이지, POP, OST, 발라드, 록/메탈..."
2,76951,17,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[까페, 잔잔한]",2017-08-28 07:09:34.000,"[포크/블루스, 랩/힙합, 재즈, 일렉트로니카, R&B/Soul, 발라드, 록/메탈..."
3,147456,33,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",2019-12-05 15:15:18.000,"[성인가요, 포크/블루스, 재즈, 어린이/태교, POP, R&B/Soul, 뉴에이지..."
4,27616,9,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",[댄스],2011-10-25 13:54:56.000,"[랩/힙합, R&B/Soul, 발라드, 댄스, 록/메탈, 아이돌]"
5,69252,435,2017 Pop Trend,"[418694, 222305, 96545, 135950, 304687, 457451...","[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...",2017-09-15 15:59:26.000,"[랩/힙합, 일렉트로니카, R&B/Soul, POP, OST, 록/메탈, EDM]"
6,45339,24,짝사랑..고백..사랑..이별..슬픔.. 감성을 자극하는곡들!,"[528130, 505036, 398947, 627363, 37748, 376358...","[짝사랑, 취향저격, 슬픔, 고백, 사랑, 이별]",2016-10-24 21:30:17.000,"[포크/블루스, R&B/Soul, OST, 발라드, 록/메탈, 아이돌, 인디음악]"
7,36557,5,멍청이.. 내맘도 몰라.,"[496913, 632529, 501426, 515574, 411161, 10341...","[잔잔한, 추억, 회상]",2008-09-23 22:32:02.000,"[랩/힙합, R&B/Soul, OST, 발라드, 록/메탈, 댄스, 아이돌]"
8,70741,0,DANCING IN THE MOON-LIGHT .01,"[634861, 270738, 163936, 692209, 449477, 56342...","[일렉트로니카, 포크, 메탈, 락, 댄스, 인디]",2019-11-30 21:17:59.000,"[성인가요, 일렉트로니카, OST, 발라드, 록/메탈, 댄스, 아이돌]"
9,10288,7,[록/메탈] Written by 이일우,"[613315, 439294, 230806, 497932, 113479, 93506...","[록, Metal, 이일우, M에센셜, 메탈, Rock, 락]",2016-06-17 18:23:54.000,"[록/메탈, 인디음악]"


In [120]:
model_play.wv.vectors # 가중치 값 보기

array([[-0.5344601 ,  0.21822175,  0.29349607, ..., -0.4940652 ,
        -0.1701847 , -0.75554144],
       [-0.24406256, -0.05841425, -0.07398453, ..., -1.0890313 ,
         0.00898774, -0.5399734 ],
       [-0.99523723,  0.02168699,  0.01518068, ..., -0.19914006,
         0.3940525 , -0.5004863 ],
       ...,
       [-0.49982142,  0.7855709 , -0.2701544 , ..., -0.30409923,
         0.31602624, -0.02956222],
       [-0.56246597, -0.01886685, -0.34003285, ..., -0.16042171,
        -0.07693619, -0.83977747],
       [-1.1292807 , -0.07890259,  0.2648496 , ..., -0.21141444,
        -0.02754082, -0.20333733]], dtype=float32)

In [105]:
word_vectors = model_play.wv
vocabs = word_vectors.vocab.keys()
word_vectors_list = [word_vectors[v] for v in vocabs] # word2vec에 들어있는 단어 보여줌

# 두 단어의 거리(비슷한 정도) 계산
print(word_vectors.similarity(w1='새벽',w2='사랑'))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


0.47946393
